In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table

#  Detection utilities
def robust_bg_sigma(image, bad_value=3421.0):
    flat = image.ravel()
    flat = flat[np.isfinite(flat) & (flat != bad_value)]
    med = np.median(flat)
    mad = np.median(np.abs(flat - med))
    sig = 1.4826 * mad
    return float(med), float(sig)

def flood_fill_region(image, seed_yx, thresh, mask):
    ny, nx = image.shape
    sy, sx = seed_yx
    if sy < 0 or sy >= ny or sx < 0 or sx >= nx:
        return []
    if mask[sy, sx] or image[sy, sx] < thresh:
        return []

    region = []
    stack = [(sy, sx)]
    visited = set()

    nbrs = [(-1,-1), (-1,0), (-1,1),
            ( 0,-1),         ( 0, 1),
            ( 1,-1), ( 1,0), ( 1, 1)]

    while stack:
        y, x = stack.pop()
        if (y, x) in visited:
            continue
        visited.add((y, x))

        if y < 0 or y >= ny or x < 0 or x >= nx:
            continue
        if mask[y, x]:
            continue
        if image[y, x] < thresh:
            continue

        region.append((y, x))
        for dy, dx in nbrs:
            stack.append((y + dy, x + dx))

    return region

def detect_sources_brightest_first(image, bad_value=3421.0, n_sigma=5.0, r_mask=8, max_sources=50000):
    """
    Brightest-first detection with a boolean mask.
    Returns: cat(list of dict), mask(bool image), stats(dict)
    """
    ny, nx = image.shape

    mask = ~np.isfinite(image) | (image == bad_value)
    bg, sig = robust_bg_sigma(image, bad_value=bad_value)
    thresh = bg + n_sigma * sig

    work = image.copy()
    work[mask] = -np.inf

    yy, xx = np.ogrid[:ny, :nx]
    cat = []

    for _ in range(max_sources):
        idx = int(np.argmax(work))
        peak = float(work.flat[idx])
        if (not np.isfinite(peak)) or (peak < thresh):
            break

        y0, x0 = np.unravel_index(idx, work.shape)

        region = flood_fill_region(image, (y0, x0), thresh=thresh, mask=mask)
        if len(region) == 0:
            mask[y0, x0] = True
            work[y0, x0] = -np.inf
            continue

        ry = np.fromiter((p[0] for p in region), dtype=int)
        rx = np.fromiter((p[1] for p in region), dtype=int)
        vals = image[ry, rx]

        w = np.clip(vals - bg, 0, None)
        if np.sum(w) > 0:
            cy = float(np.sum(ry * w) / np.sum(w))
            cx = float(np.sum(rx * w) / np.sum(w))
        else:
            cy, cx = float(y0), float(x0)

        cat.append({
            "x_peak": int(x0), "y_peak": int(y0),
            "x_centroid": cx, "y_centroid": cy,
            "peak": float(image[y0, x0]),
            "npix": int(len(region)),
        })

        mask[ry, rx] = True
        rr = (yy - y0)**2 + (xx - x0)**2
        mask |= (rr <= r_mask**2)

        work[mask] = -np.inf

    return cat, mask, {"bg": bg, "sigma": sig, "thresh": thresh}

# Aperture photometry
def circular_aperture_photometry(
    image, x0, y0,
    r_ap, r_in, r_out,
    bad_value=3421.0,
    global_mask=None,
    bg_method="median",
    sigma_clip=3.0,
    clip_iters=3
):
    ny, nx = image.shape
    x0 = float(x0); y0 = float(y0)

    x_min = max(int(np.floor(x0 - r_out - 1)), 0)
    x_max = min(int(np.ceil (x0 + r_out + 1)), nx - 1)
    y_min = max(int(np.floor(y0 - r_out - 1)), 0)
    y_max = min(int(np.ceil (y0 + r_out + 1)), ny - 1)

    cut = image[y_min:y_max+1, x_min:x_max+1]
    yy, xx = np.indices(cut.shape)
    xx = xx + x_min
    yy = yy + y_min

    rr2 = (xx - x0)**2 + (yy - y0)**2
    ap_mask  = rr2 <= r_ap**2
    ann_mask = (rr2 >= r_in**2) & (rr2 <= r_out**2)

    valid = np.isfinite(cut) & (cut != bad_value)
    if global_mask is not None:
        valid &= ~global_mask[y_min:y_max+1, x_min:x_max+1]

    ap_valid  = ap_mask  & valid
    ann_valid = ann_mask & valid

    n_ap = int(np.sum(ap_valid))
    n_bg = int(np.sum(ann_valid))

    if n_ap < max(5, int(0.25 * np.pi * r_ap**2)):
        return {"ok": False, "reason": "too_few_aperture_pixels", "n_ap": n_ap, "n_bg": n_bg}

    if n_bg < 20:
        return {"ok": False, "reason": "too_few_background_pixels", "n_ap": n_ap, "n_bg": n_bg}

    ap_vals = cut[ap_valid]
    bg_vals = cut[ann_valid]

    # sigma-clipping on background annulus
    bg_work = bg_vals
    for _ in range(clip_iters):
        med = np.median(bg_work)
        mad = np.median(np.abs(bg_work - med))
        sig = 1.4826 * mad if mad > 0 else np.std(bg_work)
        if sig == 0:
            break
        keep = np.abs(bg_work - med) <= sigma_clip * sig
        if np.all(keep):
            break
        bg_work = bg_work[keep]
        if bg_work.size < 20:
            break

    bg_per_pix = float(np.mean(bg_work)) if bg_method == "mean" else float(np.median(bg_work))

    flux_ap_sum = float(np.sum(ap_vals))
    flux_net = flux_ap_sum - bg_per_pix * n_ap


    flux_err = np.nan

    return {
        "ok": True,
        "x": x0, "y": y0,
        "r_ap": float(r_ap), "r_in": float(r_in), "r_out": float(r_out),
        "n_ap": n_ap, "n_bg": n_bg,
        "bg_per_pix": bg_per_pix,
        "flux_ap_sum": flux_ap_sum,
        "flux_net": flux_net,
        "flux_err": flux_err,
    }

def flux_to_mag(flux_net, magzpt):
    if flux_net is None or not np.isfinite(flux_net) or flux_net <= 0:
        return np.nan
    return float(magzpt - 2.5 * np.log10(flux_net))

def fluxerr_to_magerr(flux_net, flux_err):
    if flux_err is None or not np.isfinite(flux_err) or flux_net is None or flux_net <= 0:
        return np.nan
    return float(1.0857362047581294 * (flux_err / flux_net))


# Sanitize results for FITS writing
def sanitize_results_for_fits(results):
    float_cols = {
        "x_centroid","y_centroid","x","y","peak",
        "r_ap","r_in","r_out",
        "bg_per_pix","flux_ap_sum","flux_net","flux_err",
        "mag","mag_err","mag_err_tot",
    }
    int_cols = {"x_peak","y_peak","npix","n_ap","n_bg"}
    bool_cols = {"ok","is_saturated"}

    clean = []
    for row in results:
        r = dict(row)

        for k in float_cols:
            if k in r:
                v = r[k]
                r[k] = np.nan if v is None else float(v)

        for k in int_cols:
            if k in r:
                v = r[k]
                if v is None or (isinstance(v, float) and np.isnan(v)):
                    r[k] = -1
                else:
                    r[k] = int(v)

        for k in bool_cols:
            if k in r:
                v = r[k]
                r[k] = False if v is None else bool(v)

        # any remaining None -> NaN
        for k, v in list(r.items()):
            if v is None:
                r[k] = np.nan

        clean.append(r)

    return clean


# Run pipeline and write FITS outputs
with fits.open("mosaic.fits") as hdul:
    img = hdul[0].data.astype(float)
    hdr = hdul[0].header

MAGZPT = hdr.get("MAGZPT", None)
MAGZRR = hdr.get("MAGZRR", None)

# Detection
cat, det_mask, stats = detect_sources_brightest_first(img, bad_value=3421.0, n_sigma=5.0, r_mask=8)
print(f"Detected {len(cat)} sources. Detection stats: {stats}")

# Photometry setup
pixscale = 0.258
r_ap = (1.5 / pixscale)
r_in  = r_ap + 3.0
r_out = r_ap + 8.0

# Photometry
results = []
for obj in cat:
    x0 = obj.get("x_centroid", obj["x_peak"])
    y0 = obj.get("y_centroid", obj["y_peak"])

    phot = circular_aperture_photometry(
        img, x0, y0,
        r_ap=r_ap, r_in=r_in, r_out=r_out,
        bad_value=3421.0,
        global_mask=None,
        bg_method="median",
        sigma_clip=3.0,
        clip_iters=3
    )

    if phot.get("ok", False) and (MAGZPT is not None):
        phot["mag"] = flux_to_mag(phot["flux_net"], MAGZPT)
        phot["mag_err"] = fluxerr_to_magerr(phot["flux_net"], phot["flux_err"])
        if MAGZRR is not None and np.isfinite(phot["mag_err"]):
            phot["mag_err_tot"] = float(np.sqrt(phot["mag_err"]**2 + float(MAGZRR)**2))
        else:
            phot["mag_err_tot"] = np.nan
    else:
        phot["mag"] = np.nan
        phot["mag_err"] = np.nan
        phot["mag_err_tot"] = np.nan

    results.append({**obj, **phot})

print("Example row:", results[0])

out_dir = os.path.join(os.getcwd(), "outputs")
os.makedirs(out_dir, exist_ok=True)

out_img = os.path.join(out_dir, "mosaic_photometry_image.fits")
out_cat = os.path.join(out_dir, "galaxy_catalog_photometry.fits")

# (A) Save the image used for photometry
fits.PrimaryHDU(data=img.astype("float32"), header=hdr).writeto(out_img, overwrite=True)
print("Wrote image FITS:", out_img)

# (B) Save the catalogue as FITS binary table
clean_results = sanitize_results_for_fits(results)
tab = Table(rows=clean_results)
tab.write(out_cat, overwrite=True)
print("Wrote catalogue FITS:", out_cat)
print("Table columns:", tab.colnames)


Detected 5791 sources. Detection stats: {'bg': 3419.0, 'sigma': 14.825999999999999, 'thresh': 3493.13}
Example row: {'x_peak': 2530, 'y_peak': 657, 'x_centroid': 2530.0, 'y_centroid': 657.0, 'peak': 65535.0, 'npix': 1, 'ok': True, 'x': 2530.0, 'y': 657.0, 'r_ap': 5.813953488372093, 'r_in': 8.813953488372093, 'r_out': 13.813953488372093, 'n_ap': 101, 'n_bg': 344, 'bg_per_pix': 3405.0, 'flux_ap_sum': 406257.0, 'flux_net': 62352.0, 'flux_err': nan, 'mag': 13.312874028878463, 'mag_err': nan, 'mag_err_tot': nan}
Wrote image FITS: C:\Users\wz2523\OneDrive - Imperial College London\Lab Astro\Astro\Astro\Fits_Data\outputs\mosaic_photometry_image.fits
Wrote catalogue FITS: C:\Users\wz2523\OneDrive - Imperial College London\Lab Astro\Astro\Astro\Fits_Data\outputs\galaxy_catalog_photometry.fits
Table columns: ['x_peak', 'y_peak', 'x_centroid', 'y_centroid', 'peak', 'npix', 'ok', 'x', 'y', 'r_ap', 'r_in', 'r_out', 'n_ap', 'n_bg', 'bg_per_pix', 'flux_ap_sum', 'flux_net', 'flux_err', 'mag', 'mag_err